In [ ]:
# default_exp data.question_answering


In [ ]:
# all_slow


In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# data.question_answering

> Question/Answering tasks are models that require two text inputs (a context that includes the answer and the question).  The objective is to predict the start/end tokens of the answer in the context). This module contains the bits required to use the fastai DataBlock API and/or mid-level data processing pipelines to organize your data for question/answering tasks.

In [ ]:
# export
import ast
from functools import reduce

from datasets import Dataset
from fastcore.all import *
from fastai.data.block import DataBlock, CategoryBlock, ColReader, ColSplitter
from fastai.imports import *
from fastai.losses import CrossEntropyLossFlat
from fastai.torch_core import *
from fastai.torch_imports import *
from transformers import AutoModelForQuestionAnswering, logging, PretrainedConfig, PreTrainedTokenizerBase, PreTrainedModel

from blurr.utils import BLURR
from blurr.data.core import HF_BaseInput, HF_AfterBatchTransform, HF_BeforeBatchTransform, Preprocessor, first_blurr_tfm

logging.set_verbosity_error()


In [ ]:
# hide_input
import pdb

from datasets import load_dataset
from fastai.data.core import DataLoader, DataLoaders, TfmdDL
from fastai.data.external import untar_data, URLs
from fastai.data.transforms import *
from fastcore.test import *
from nbverbose.showdoc import show_doc

from blurr.utils import print_versions
from blurr.data.core import HF_TextBlock

os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option('display.max_colwidth', 100)

print("What we're running with at the time this documentation was generated:")
print_versions("torch fastai transformers")


What we're running with at the time this documentation was generated:
torch: 1.10.1+cu111
fastai: 2.5.3
transformers: 4.15.0


In [ ]:
# hide
# cuda
torch.cuda.set_device(1)
print(f"Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}")


Using GPU #1: GeForce GTX 1080 Ti


## Setup

We'll use a subset of `squad_v2` to demonstrate how to configure your blurr code for extractive question answering

In [ ]:
train_ds = load_dataset("squad_v2", split='train[:1000]')

Reusing dataset squad_v2 (/home/wgilliam/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


In [ ]:
squad_df = pd.DataFrame(train_ds)

print(len(squad_df))
squad_df.head(2)


1000


,id,title,context,question,answers
0,56be85543aeaaa14008c9063,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an America...",When did Beyonce start becoming popular?,"{'text': ['in the late 1990s'], 'answer_start': [269]}"
1,56be85543aeaaa14008c9065,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an America...",What areas did Beyonce compete in when she was growing up?,"{'text': ['singing and dancing'], 'answer_start': [207]}"


In [ ]:
model_cls = AutoModelForQuestionAnswering

pretrained_model_name = "roberta-base"  #'xlm-mlm-ende-1024'
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, model_cls=model_cls)

max_seq_len = 128
vocab = dict(enumerate(range(max_seq_len)))


## Utility methods

Starting with version 2.0.0, blurr provides a `find_answer_token_idxs` method that can be used in preprocessing your raw dataset beforehand, or on the fly.  It returns tensors indicating the start/end token indicies and one indicating whether the answer can be found in the provided set of `input_ids`.

In [ ]:
# export
def find_answer_token_idxs(hf_tokenizer, ans_data, input_ids, offset_mapping, qst_mask):
    # mask the question tokens so they aren't included in the search
    masked_offset_mapping = offset_mapping.clone()
    masked_offset_mapping[qst_mask] = tensor([-100, -100])

    # based on the character start/end index, see if we can find the span of tokens in the `offset_mapping`
    starts = torch.where((masked_offset_mapping[:, 0] == ans_data[1]) | (masked_offset_mapping[:, 1] == ans_data[1]))[0]
    ends = torch.where((masked_offset_mapping[:, 0] <= ans_data[2]) & (masked_offset_mapping[:, 1] >= ans_data[2]))[0]

    if len(starts) > 0 and len(ends) > 0:
        start, end = starts[-1], ends[-1]
        for s, e in itertools.product(starts, ends):
            txt = hf_tokenizer.decode(input_ids[s:e])
            if txt.strip() == ans_data[0].strip():
                start, end = s, e
                break

        if end < len(masked_offset_mapping):
            return (start, end, tensor(1))

    # if neither star or end is found, or the end token is part of this chunk, consider the answer not found
    return (tensor(0), tensor(0), tensor(0))


In [ ]:
show_doc(find_answer_token_idxs)


<h4 id="find_answer_token_idxs" class="doc_header"><code>find_answer_token_idxs</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>find_answer_token_idxs</code>(**`hf_tokenizer`**, **`ans_data`**, **`input_ids`**, **`offset_mapping`**, **`qst_mask`**)



**Parameters:**


 - **`hf_tokenizer`** : *`<class 'inspect._empty'>`*

 - **`ans_data`** : *`<class 'inspect._empty'>`*

 - **`input_ids`** : *`<class 'inspect._empty'>`*

 - **`offset_mapping`** : *`<class 'inspect._empty'>`*

 - **`qst_mask`** : *`<class 'inspect._empty'>`*


## Preprocessing

With version 2.0.0 of blurr, we include a `Preprocessor` for question answering that can either truncate texts or else chunk long documents into multiple examples

In [ ]:
# export
class QuestionAnsweringPreprocessor(Preprocessor):
    def __init__(
        self,
        # A Hugging Face tokenizer
        hf_tokenizer: PreTrainedTokenizerBase,
        # The number of examples to process at a time
        batch_size: int = 1000,
        # The unique identifier in the dataset. If not specified and "return_overflowing_tokens": True, an "_id" attribute
        # will be added to your dataset with its value a unique, sequential integer, assigned to each record
        id_attr: Optional[str] = None,
        # The attribute in your dataset that contains the context (where the answer is included) (default: 'context')
        ctx_attr: str = "context",
        # The attribute in your dataset that contains the question being asked (default: 'question')
        qst_attr: str = "question",
        # The attribute in your dataset that contains the actual answer (default: 'answer_text')
        ans_attr: str = "answer_text",
        # The attribute in your dataset that contains the actual answer (default: 'answer_text')
        ans_start_char_idx: str = "ans_start_char_idx",
        # The attribute in your dataset that contains the actual answer (default: 'answer_text')
        ans_end_char_idx: str = "ans_end_char_idx",
        # The attribute that should be created if your are processing individual training and validation
        # datasets into a single dataset, and will indicate to which each example is associated
        is_valid_attr: Optional[str] = "is_valid",
        # Tokenization kwargs that will be applied with calling the tokenizer (default: {"return_overflowing_tokens": True})
        tok_kwargs: dict = {"return_overflowing_tokens": True},
    ):
        # these values are mandatory
        tok_kwargs["return_offsets_mapping"] = True                 # allows us to map tokens -> raw characters
        tok_kwargs["padding"] = tok_kwargs.get("padding", True) 
        tok_kwargs["return_tensors"] = "pt"

         # shift the question and context appropriately based on the tokenizers padding strategy
        if hf_tokenizer.padding_side == "right":
            tok_kwargs["truncation"] = "only_second"
            text_attrs = [qst_attr, ctx_attr]
        else:
            tok_kwargs["truncation"] = "only_first"
            text_attrs = [ctx_attr, qst_attr]

        super().__init__(hf_tokenizer, batch_size, text_attrs=text_attrs, tok_kwargs=tok_kwargs)

        self.id_attr = id_attr
        self.qst_attr, self.ctx_attr = qst_attr, ctx_attr
        self.ans_attr, self.ans_start_char_idx, self.ans_end_char_idx = ans_attr, ans_start_char_idx, ans_end_char_idx
        self.is_valid_attr = is_valid_attr

    def process_df(self, training_df: pd.DataFrame, validation_df: Optional[pd.DataFrame] = None):
        df = super().process_df(training_df, validation_df)

        # a unique Id for each example is required to properly score question answering results when chunking long
        # documents (e.g., return_overflowing_tokens=True)

        if self.id_attr is None and self.tok_kwargs.get("return_overflowing_tokens", False):
            df.insert(0, "_id", range(len(df)))

        proc_data = []
        for row_idx, row in df.iterrows():
            # fetch data elements required to build a modelable dataset
            inputs = self._tokenize_function(row)
            ans_text, start_char_idx, end_char_idx = row[self.ans_attr], row[self.ans_start_char_idx], row[self.ans_end_char_idx] + 1
            ans_data = (ans_text, start_char_idx, end_char_idx)

            # if "return_overflowing_tokens = True", our BatchEncoding will include an "overflow_to_sample_mapping" list
            overflow_mapping = inputs["overflow_to_sample_mapping"] if ("overflow_to_sample_mapping" in inputs) else [0]

            for idx in range(len(overflow_mapping)):
                # update the targets: is_found (s[1]), answer start token index (s[2]), and answer end token index (s[3])
                qst_mask = [i != 1 if self.hf_tokenizer.padding_side == "right" else i != 0 for i in inputs.sequence_ids(idx)]
                start, end, has_ans = find_answer_token_idxs(hf_tokenizer, ans_data, inputs["input_ids"][idx], inputs["offset_mapping"][idx], qst_mask)

                overflow_row = row.copy()
                overflow_row[self.ans_end_char_idx] = end_char_idx
                overflow_row["has_answer"] = has_ans.item()
                overflow_row["ans_start_token_idx"] = start.item()
                overflow_row["ans_end_token_idx"] = end.item()

                for k in inputs.keys():
                    overflow_row[k] = inputs[k][idx].numpy()
                
                proc_data.append(overflow_row)

        return pd.DataFrame(proc_data)

    def process_hf_dataset(self, training_ds: Dataset, validation_ds: Optional[Dataset] = None):
        ds = super().process_hf_dataset(training_ds, validation_ds)

        # return the pre-processed DataFrame
        return ds


How to preprocess your data

In [ ]:
squad_df["ans_start_char_idx"] = squad_df.answers.apply(lambda v: v["answer_start"][0])
squad_df["answer_text"] = squad_df.answers.apply(lambda v: v["text"][0])
squad_df["ans_end_char_idx"] = squad_df["ans_start_char_idx"].astype(int) + squad_df["answer_text"].str.len()

print(len(squad_df))
squad_df.head(2)


1000


,id,title,context,question,answers,ans_start_char_idx,answer_text,ans_end_char_idx
0,56be85543aeaaa14008c9063,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an America...",When did Beyonce start becoming popular?,"{'text': ['in the late 1990s'], 'answer_start': [269]}",269,in the late 1990s,286
1,56be85543aeaaa14008c9065,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an America...",What areas did Beyonce compete in when she was growing up?,"{'text': ['singing and dancing'], 'answer_start': [207]}",207,singing and dancing,226


In [ ]:
tok_kwargs = {"return_overflowing_tokens": True, "max_length": max_seq_len, "stride": 64}
preprocessor = QuestionAnsweringPreprocessor(hf_tokenizer, id_attr="id", tok_kwargs=tok_kwargs)
proc_df = preprocessor.process_df(squad_df)

print(len(proc_df))
proc_df.head()

3109


,id,title,context,question,answers,ans_start_char_idx,answer_text,ans_end_char_idx,has_answer,ans_start_token_idx,ans_end_token_idx,input_ids,attention_mask,offset_mapping,overflow_to_sample_mapping
0,56be85543aeaaa14008c9063,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an America...",When did Beyonce start becoming popular?,"{'text': ['in the late 1990s'], 'answer_start': [269]}",269,in the late 1990s,287,1,84,89,"[0, 520, 222, 12674, 1755, 386, 1959, 1406, 116, 2, 2, 12674, 12695, 272, 354, 6591, 10690, 1634...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0], [0, 4], [5, 8], [9, 14], [14, 16], [17, 22], [23, 31], [32, 39], [39, 40], [0, 0], [0, ...",0
0,56be85543aeaaa14008c9063,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an America...",When did Beyonce start becoming popular?,"{'text': ['in the late 1990s'], 'answer_start': [269]}",269,in the late 1990s,287,1,32,37,"[0, 520, 222, 12674, 1755, 386, 1959, 1406, 116, 2, 2, 11, 2499, 6, 1184, 6, 79, 3744, 11, 1337,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0], [0, 4], [5, 8], [9, 14], [14, 16], [17, 22], [23, 31], [32, 39], [39, 40], [0, 0], [0, ...",0
0,56be85543aeaaa14008c9063,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an America...",When did Beyonce start becoming popular?,"{'text': ['in the late 1990s'], 'answer_start': [269]}",269,in the late 1990s,287,0,0,0,"[0, 520, 222, 12674, 1755, 386, 1959, 1406, 116, 2, 2, 333, 1059, 65, 9, 5, 232, 18, 275, 12, 11...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0], [0, 4], [5, 8], [9, 14], [14, 16], [17, 22], [23, 31], [32, 39], [39, 40], [0, 0], [0, ...",0
1,56be85543aeaaa14008c9065,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an America...",What areas did Beyonce compete in when she was growing up?,"{'text': ['singing and dancing'], 'answer_start': [207]}",207,singing and dancing,227,1,77,80,"[0, 653, 911, 222, 12674, 1755, 3511, 11, 77, 79, 21, 1197, 62, 116, 2, 2, 12674, 12695, 272, 35...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0], [0, 4], [5, 10], [11, 14], [15, 20], [20, 22], [23, 30], [31, 33], [34, 38], [39, 42], ...",0
1,56be85543aeaaa14008c9065,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an America...",What areas did Beyonce compete in when she was growing up?,"{'text': ['singing and dancing'], 'answer_start': [207]}",207,singing and dancing,227,1,30,33,"[0, 653, 911, 222, 12674, 1755, 3511, 11, 77, 79, 21, 1197, 62, 116, 2, 2, 3390, 4, 8912, 8, 117...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0], [0, 4], [5, 10], [11, 14], [15, 20], [20, 22], [23, 30], [31, 33], [34, 38], [39, 42], ...",0


In [ ]:
sampled_df = proc_df.sample(n=10)
for row_idx, row in sampled_df.iterrows():
    test_example = row

    if test_example.has_answer == 1:
        test_eq(
            test_example.answer_text,
            hf_tokenizer.decode(test_example["input_ids"][test_example.ans_start_token_idx : test_example.ans_end_token_idx]).strip(),
        )
    else:
        test_eq(test_example.ans_start_token_idx, 0)
        test_eq(test_example.ans_end_token_idx, 0)


If you want to remove texts longer than your model will hold (and include only answerable contexts)

In [ ]:
preprocessor = QuestionAnsweringPreprocessor(hf_tokenizer, tok_kwargs={"return_overflowing_tokens": False, "max_length": max_seq_len})
proc2_df = preprocessor.process_df(squad_df)
proc2_df = proc2_df[(proc2_df.ans_end_token_idx < max_seq_len) & (proc2_df.has_answer == 1)]

print(len(proc2_df))
proc2_df.head()


731


,id,title,context,question,answers,ans_start_char_idx,answer_text,ans_end_char_idx,has_answer,ans_start_token_idx,ans_end_token_idx,input_ids,attention_mask,offset_mapping
0,56be85543aeaaa14008c9063,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an America...",When did Beyonce start becoming popular?,"{'text': ['in the late 1990s'], 'answer_start': [269]}",269,in the late 1990s,287,1,84,89,"[0, 520, 222, 12674, 1755, 386, 1959, 1406, 116, 2, 2, 12674, 12695, 272, 354, 6591, 10690, 1634...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0], [0, 4], [5, 8], [9, 14], [14, 16], [17, 22], [23, 31], [32, 39], [39, 40], [0, 0], [0, ..."
1,56be85543aeaaa14008c9065,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an America...",What areas did Beyonce compete in when she was growing up?,"{'text': ['singing and dancing'], 'answer_start': [207]}",207,singing and dancing,227,1,77,80,"[0, 653, 911, 222, 12674, 1755, 3511, 11, 77, 79, 21, 1197, 62, 116, 2, 2, 12674, 12695, 272, 35...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0], [0, 4], [5, 10], [11, 14], [15, 20], [20, 22], [23, 30], [31, 33], [34, 38], [39, 42], ..."
3,56bf6b0f3aeaaa14008c9601,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an America...",In what city and state did Beyonce grow up?,"{'text': ['Houston, Texas'], 'answer_start': [166]}",166,"Houston, Texas",181,1,69,72,"[0, 96, 99, 343, 8, 194, 222, 12674, 1755, 1437, 1733, 62, 116, 1437, 2, 2, 12674, 12695, 272, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0], [0, 2], [3, 7], [8, 12], [13, 16], [17, 22], [23, 26], [27, 32], [32, 34], [35, 35], [3..."
4,56bf6b0f3aeaaa14008c9602,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an America...",In which decade did Beyonce become famous?,"{'text': ['late 1990s'], 'answer_start': [276]}",276,late 1990s,287,1,87,90,"[0, 96, 61, 2202, 222, 12674, 1755, 555, 3395, 116, 2, 2, 12674, 12695, 272, 354, 6591, 10690, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0], [0, 2], [3, 8], [9, 15], [16, 19], [20, 25], [25, 27], [28, 34], [35, 41], [41, 42], [0..."
5,56bf6b0f3aeaaa14008c9603,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an America...",In what R&B group was she the lead singer?,"{'text': ['Destiny's Child'], 'answer_start': [320]}",320,Destiny's Child,336,1,103,106,"[0, 96, 99, 248, 947, 387, 333, 21, 79, 5, 483, 3250, 116, 2, 2, 12674, 12695, 272, 354, 6591, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0], [0, 2], [3, 7], [8, 9], [9, 10], [10, 11], [12, 17], [18, 21], [22, 25], [26, 29], [30,..."


## Mid-level API

### `HF_QuestionAnswerInput`

In [ ]:
# export
class HF_QuestionAnswerInput(HF_BaseInput):
    pass


### `HF_QABeforeBatchTransform`

In [ ]:
# export
class HF_QABeforeBatchTransform(HF_BeforeBatchTransform):
    def __init__(
        self,
        # The abbreviation/name of your Hugging Face transformer architecture (e.b., bert, bart, etc..)
        hf_arch: str,
        # A specific configuration instance you want to use
        hf_config: PretrainedConfig,
        # A Hugging Face tokenizer
        hf_tokenizer: PreTrainedTokenizerBase,
        # A Hugging Face model
        hf_model: PreTrainedModel,
        # If you are passing in the "input_ids" as your inputs, set `is_pretokenized` = True
        is_pretokenized: bool = False,
        # The token ID that should be ignored when calculating the loss
        ignore_token_id=CrossEntropyLossFlat().ignore_index,
        # To control the length of the padding/truncation. It can be an integer or None,
        # in which case it will default to the maximum length the model can accept. If the model has no
        # specific maximum input length, truncation/padding to max_length is deactivated.
        # See [Everything you always wanted to know about padding and truncation](https://huggingface.co/transformers/preprocessing.html#everything-you-always-wanted-to-know-about-padding-and-truncation)
        max_length: int = None,
        # To control the `padding` applied to your `hf_tokenizer` during tokenization. If None, will default to
        # `False` or `'do_not_pad'.
        # See [Everything you always wanted to know about padding and truncation](https://huggingface.co/transformers/preprocessing.html#everything-you-always-wanted-to-know-about-padding-and-truncation)
        padding: Union[bool, str] = True,
        # To control `truncation` applied to your `hf_tokenizer` during tokenization. If None, will default to
        # `False` or `do_not_truncate`.
        # See [Everything you always wanted to know about padding and truncation](https://huggingface.co/transformers/preprocessing.html#everything-you-always-wanted-to-know-about-padding-and-truncation)
        truncation: Union[bool, str] = "only_second",
        # The `is_split_into_words` argument applied to your `hf_tokenizer` during tokenization. Set this to `True`
        # if your inputs are pre-tokenized (not numericalized)
        is_split_into_words: bool = False,
        # Any other keyword arguments you want included when using your `hf_tokenizer` to tokenize your inputs
        tok_kwargs: dict = {},
        # Keyword arguments to apply to `HF_BeforeBatchTransform`
        **kwargs
    ):

        # "return_special_tokens_mask" and "return_offsets_mapping" are mandatory for extractive QA in blurr
        tok_kwargs = { **tok_kwargs, **{"return_special_tokens_mask": True, "return_offsets_mapping": True}}

        super().__init__(
            hf_arch,
            hf_config,
            hf_tokenizer,
            hf_model,
            is_pretokenized=is_pretokenized,
            ignore_token_id=ignore_token_id,
            max_length=max_length,
            padding=padding,
            truncation=truncation,
            is_split_into_words=is_split_into_words,
            tok_kwargs=tok_kwargs,
            **kwargs
        )

    def encodes(self, samples):
        samples, batch_encoding = super().encodes(samples, return_batch_encoding=True)

        if self.is_pretokenized:
            return samples

        updated_samples = []
        for idx, s in enumerate(samples):
            # update the targets: is_found (s[1]), answer start token index (s[2]), and answer end token index (s[3])
            qst_mask = [i != 1 for i in batch_encoding.sequence_ids(idx)]
            start, end, has_ans = find_answer_token_idxs(self.hf_tokenizer, s[1], s[0]["input_ids"], s[0]["offset_mapping"], qst_mask)
            start_t, end_t, has_ans_t = TensorCategory(start), TensorCategory(end), TensorCategory(has_ans)

            # cls_index: location of CLS token (used by xlnet and xlm); is a list.index(value) for pytorch tensor's
            s[0]["cls_index"] = (s[0]["input_ids"] == self.hf_tokenizer.cls_token_id).nonzero()[0]
            # p_mask: mask with 1 for token than cannot be in the answer, else 0 (used by xlnet and xlm)
            s[0]["p_mask"] = s[0]["special_tokens_mask"]

            updated_samples.append((s[0], has_ans_t, start_t, end_t))

        return updated_samples


In [ ]:
# export
def get_dummy_token_idx(r):
    return 0


## Examples

The following eamples demonstrate several approaches to construct your `DataBlock` for question answering tasks using the mid-level API

### Using the mid-level API

#### Batch-Time Tokenization

##### Step 1: Get your Hugging Face objects.

In [ ]:
pretrained_model_name = "distilroberta-base"
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, model_cls=AutoModelForQuestionAnswering)

max_seq_len = 128
vocab = dict(enumerate(range(max_seq_len)))

#####  Step 2: Create your `DataBlock`

In [ ]:
before_batch_tfm = HF_QABeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model, max_length=max_seq_len)

blocks = (
    HF_TextBlock(before_batch_tfm=before_batch_tfm, input_return_type=HF_QuestionAnswerInput),
    None,
    CategoryBlock(vocab=vocab),
    CategoryBlock(vocab=vocab),
)

def get_ans_and_start_end_char_idxs(r):
    start = r.answers["answer_start"][0]
    answer_text = r.answers["text"][0]
    end = start + len(answer_text) + 1
    return (answer_text, start, end)


dblock = DataBlock(
    blocks=blocks,
    get_x=lambda x: (x.question, x.context),
    get_y=[get_ans_and_start_end_char_idxs, get_dummy_token_idx, get_dummy_token_idx],
    splitter=RandomSplitter(),
    n_inp=1,
)


##### Step 3: Build your `DataLoaders`

In [ ]:
dls = dblock.dataloaders(squad_df, bs=4)
len(dls.train), len(dls.valid)


(200, 50)

In [ ]:
b = dls.one_batch()
len(b), len(b[0]), len(b[1]), len(b[2])


(4, 6, 4, 4)

In [ ]:
b[0]["input_ids"].shape, b[0]["attention_mask"].shape, b[1].shape, b[2].shape


(torch.Size([4, 128]), torch.Size([4, 128]), torch.Size([4]), torch.Size([4]))

In [ ]:
# export
@typedispatch
def show_batch(
    # This typedispatched `show_batch` will be called for `HF_QuestionAnswerInput` typed inputs
    x: HF_QuestionAnswerInput,
    # Your targets
    y,
    # Your raw inputs/targets
    samples,
    # Your `DataLoaders`. This is required so as to get at the Hugging Face objects for
    # decoding them into something understandable
    dataloaders,
    # Your `show_batch` context
    ctxs=None,
    # The maximum number of items to show
    max_n=6,
    # Any truncation your want applied to your decoded inputs
    trunc_at=None,
    # Any other keyword arguments you want applied to `show_batch`
    **kwargs
):
    # grab our tokenizer
    tfm = first_blurr_tfm(dataloaders, HF_QABeforeBatchTransform)
    hf_tokenizer = tfm.hf_tokenizer

    res = L()
    for sample, input_ids, has_ans, start, end in zip(samples, x, *y):
        txt = hf_tokenizer.decode(sample[0], skip_special_tokens=True)[:trunc_at]
        found = has_ans.item() == 1
        ans_text = hf_tokenizer.decode(input_ids[start:end], skip_special_tokens=False)
        res.append((txt, found, (start.item(), end.item()), ans_text))

    display_df(pd.DataFrame(res, columns=["text", "found", "start/end", "answer"])[:max_n])
    return ctxs


The `show_batch` method above allows us to create a more interpretable view of our question/answer data.

In [ ]:
dls.show_batch(dataloaders=dls, max_n=4, trunc_at=500)


,text,found,start/end,answer
0,"Which prominent star felt the 2009 Female Video of the Year award should have went to Beyoncé instead of Taylor Swift? On April 4, 2008, Beyoncé married Jay Z. She publicly revealed their marriage in a video montage at the listening party for her third studio album, I Am... Sasha Fierce, in Manhattan's Sony Club on October 22, 2008. I Am... Sasha Fierce was released on November 18, 2008 in the United States. The album formally introduces Beyoncé's alter ego Sasha Fierce, conceived during the ma",False,"(0, 0)",
1,"The video for what song won Beyoncé the 2009 MTV Video of the Year award? On April 4, 2008, Beyoncé married Jay Z. She publicly revealed their marriage in a video montage at the listening party for her third studio album, I Am... Sasha Fierce, in Manhattan's Sony Club on October 22, 2008. I Am... Sasha Fierce was released on November 18, 2008 in the United States. The album formally introduces Beyoncé's alter ego Sasha Fierce, conceived during the making of her 2003 single ""Crazy in Love"", sell",False,"(0, 0)",
2,"Which singer beat out Beyonce for best video performance? On April 4, 2008, Beyoncé married Jay Z. She publicly revealed their marriage in a video montage at the listening party for her third studio album, I Am... Sasha Fierce, in Manhattan's Sony Club on October 22, 2008. I Am... Sasha Fierce was released on November 18, 2008 in the United States. The album formally introduces Beyoncé's alter ego Sasha Fierce, conceived during the making of her 2003 single ""Crazy in Love"", selling 482,000 copi",False,"(0, 0)",
3,"When did Beyoncé get married? On April 4, 2008, Beyoncé married Jay Z. She publicly revealed their marriage in a video montage at the listening party for her third studio album, I Am... Sasha Fierce, in Manhattan's Sony Club on October 22, 2008. I Am... Sasha Fierce was released on November 18, 2008 in the United States. The album formally introduces Beyoncé's alter ego Sasha Fierce, conceived during the making of her 2003 single ""Crazy in Love"", selling 482,000 copies in its first week, debuti",True,"(11, 15)","April 4, 2008"


#### Pre-tokenized/numericalized

##### Step 1a: Get your Hugging Face objects.

In [ ]:
pretrained_model_name = "roberta-base"  #'xlm-mlm-ende-1024'
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, model_cls=AutoModelForQuestionAnswering)

max_seq_len = 128
vocab = dict(enumerate(range(max_seq_len)))

##### Step 1b. Preprocess dataset

In [ ]:
preprocessor = QuestionAnsweringPreprocessor(
    hf_tokenizer,
    id_attr="id",
    ctx_attr="context",
    qst_attr="question",
    ans_attr="answer_text",
    ans_start_char_idx="ans_start_char_idx",
    ans_end_char_idx="ans_end_char_idx",
    tok_kwargs={"return_overflowing_tokens": True, "max_length": max_seq_len, "stride": 2},
)
proc_df = preprocessor.process_df(squad_df)
proc_df.head(2)

,id,title,context,question,answers,ans_start_char_idx,answer_text,ans_end_char_idx,has_answer,ans_start_token_idx,ans_end_token_idx,input_ids,attention_mask,offset_mapping,overflow_to_sample_mapping
0,56be85543aeaaa14008c9063,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an America...",When did Beyonce start becoming popular?,"{'text': ['in the late 1990s'], 'answer_start': [269]}",269,in the late 1990s,287,1,84,89,"[0, 520, 222, 12674, 1755, 386, 1959, 1406, 116, 2, 2, 12674, 12695, 272, 354, 6591, 10690, 1634...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0], [0, 4], [5, 8], [9, 14], [14, 16], [17, 22], [23, 31], [32, 39], [39, 40], [0, 0], [0, ...",0
0,56be85543aeaaa14008c9063,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an America...",When did Beyonce start becoming popular?,"{'text': ['in the late 1990s'], 'answer_start': [269]}",269,in the late 1990s,287,0,0,0,"[0, 520, 222, 12674, 1755, 386, 1959, 1406, 116, 2, 2, 1816, 1134, 9, 70, 86, 4, 2667, 25224, 79...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0], [0, 4], [5, 8], [9, 14], [14, 16], [17, 22], [23, 31], [32, 39], [39, 40], [0, 0], [0, ...",0


##### Step 2: Create your `DataBlock`

In [ ]:
before_batch_tfm = HF_QABeforeBatchTransform(
    hf_arch,
    hf_config,
    hf_tokenizer,
    hf_model,
    is_pretokenized=True,
    max_length=max_seq_len,
    padding="max_length",
    tok_kwargs={"add_special_tokens": False},
)

blocks = (
    HF_TextBlock(before_batch_tfm=before_batch_tfm, input_return_type=HF_QuestionAnswerInput),
    None,
    CategoryBlock(vocab=vocab),
    CategoryBlock(vocab=vocab),
)


dblock = DataBlock(
    blocks=blocks,
    get_x=ItemGetter("input_ids"),
    get_y=[ItemGetter("has_answer"), ItemGetter("ans_start_token_idx"), ItemGetter("ans_end_token_idx")],
    splitter=RandomSplitter(),
    n_inp=1,
)


##### Step 3: Build your `DataLoaders`

In [ ]:
dls = dblock.dataloaders(proc_df, bs=4)
len(dls.train), len(dls.valid)


(438, 110)

In [ ]:
b = dls.one_batch()
len(b), len(b[0]), len(b[1]), len(b[2])


(4, 3, 4, 4)

In [ ]:
b[0]["input_ids"].shape, b[0]["attention_mask"].shape, b[1].shape, b[2].shape


(torch.Size([4, 128]), torch.Size([4, 128]), torch.Size([4]), torch.Size([4]))

In [ ]:
dls.show_batch(dataloaders=dls, max_n=4)


,text,found,start/end,answer
0,"What song won Best R&B Performance in the 43 Annual Grammy Awards? Best R&B/Soul Album of the Year, Best R&B/Soul or Rap New Artist, and Best R&B/Soul Single for ""No, No, No"". The group released their multi-platinum second album The Writing's on the Wall in 1999. The record features some of the group's most widely known songs such as ""Bills, Bills, Bills"", the group's first number-one single, ""Jumpin' Jumpin'"" and ""Say My Name"", which became their most successful song at the",True,"(115, 118)",Say My Name
1,"What title did Complex award Beyoncé? cities around the world, including New York, Washington, D.C., Amsterdam, Bangkok, Hollywood and Sydney.",False,"(0, 0)",
2,"Beyonce's grandma's name was? Beyoncé and her mother introduced House of Deréon, a contemporary women's fashion line, in 2005. The concept is inspired by three generations of women in their family, the name paying tribute to Beyoncé's grandmother, Agnèz Deréon, a respected seamstress. According to Tina, the overall style of the line best reflects her and Beyoncé's taste and style. Beyoncé and her mother founded their family's company Beyond Productions, which provides the licensing and brand management for House of Deréon, and its junior collection, Deréon. House",True,"(56, 63)",Agnèz Deréon
3,At what age did Frédéric depart from Poland? November 1830 Uprising.,False,"(0, 0)",


## Summary

This module includes all the low, mid, and high-level API bits for extractive Q&A tasks data preparation.

In [ ]:
# hide
from nbdev.export import notebook2script

notebook2script()


Converted 00_utils.ipynb.
Converted 01_data-core.ipynb.
Converted 01_modeling-core.ipynb.
Converted 02_data-language-modeling.ipynb.
Converted 02_modeling-language-modeling.ipynb.
Converted 03_data-token-classification.ipynb.
Converted 03_modeling-token-classification.ipynb.
Converted 04_data-question-answering.ipynb.
Converted 04_modeling-question-answering.ipynb.
Converted 10_data-seq2seq-core.ipynb.
Converted 10_modeling-seq2seq-core.ipynb.
Converted 11_data-seq2seq-summarization.ipynb.
Converted 11_modeling-seq2seq-summarization.ipynb.
Converted 12_data-seq2seq-translation.ipynb.
Converted 12_modeling-seq2seq-translation.ipynb.
Converted 99a_examples-high-level-api.ipynb.
Converted 99b_examples-glue.ipynb.
Converted 99c_examples-glue-plain-pytorch.ipynb.
Converted 99d_examples-multilabel.ipynb.
Converted 99e_examples-causal-lm-gpt2.ipynb.
Converted index.ipynb.
